In [ ]:
import numpy as np
import scipy.special
import scipy.ndimage
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import time

%matplotlib inline

In [ ]:
class neuralNetwork:

    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        self.train_errors = [] 
        pass
    
    def train(self, inputs_list, targets_list):
        inputs  = np.array(inputs_list , ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        output_inputs = np.dot(self.who, hidden_outputs)
        output_outputs = self.activation_function(output_inputs)
        
        output_errors = targets - output_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        
        self.who += self.lr * np.dot((output_errors * output_outputs * (1 - output_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), np.transpose(inputs))
        
        self.train_errors.append(np.sum(output_errors**2)) 
    
        pass
    
    def query(self, inputs_lists):
        inputs = np.array(inputs_lists, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        output_inputs = np.dot(self.who, hidden_outputs)
        output_outputs = self.activation_function(output_inputs)
        
        return output_outputs
    
    def return_wih(self):
        return self.wih
    def return_who(self):
        return self.who
    def return_err(self):
        return self.train_errors
    

In [ ]:
try:
    f = open('MNIST_Train_extended.pckl', 'rb')
    traindf = pickle.load(f)
    f.close()
    print("found training df")
    creation_of_training_df_required = False
except:
    traindf = pd.DataFrame()
    print("training df not found creation required")
    creation_of_training_df_required = True

try:
    f = open('MNIST_Test.pckl', 'rb')
    testdf = pickle.load(f)
    f.close()
    print("found testing df")
    creation_of_testing_df_required = False
except:
    testdf = pd.DataFrame()
    print("testing df not found creation required")
    creation_of_testing_df_required = True

In [ ]:
hidden       = (5, 10, 20, 50, 100, 200, 250, 300)
LearningRate = (0.001, 0.002, 0.005, 0.01, 0.05, 0.1, 0.3)
epochs       = 20

In [ ]:
input_nodes   = 28*28
output_nodes  = 10

NNperformance = pd.DataFrame()

for h in hidden:
    hidden_nodes = h
    print('hidden layer count '+str(h))
    for L in LearningRate:
        learning_rate = L
        print('learning rate '+str(L))
        n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
        for e in range(epochs):
            start_time = time.time()
            # print('epoch '+str(e))
            for index, row in traindf.iterrows():
                input_raw  = row['image']
                input_blr  = row['blurred']
                input_rtp  = row['rotplus10']
                input_rtm  = row['rotminu10']
                targets = np.zeros(output_nodes) + 0.01
                targets[int(row['Label'])] = 0.99
                n.train(input_raw, targets)
                n.train(input_blr, targets)
                n.train(input_rtp, targets)
                n.train(input_rtm, targets)
                pass
            scorecard=[]
            for index, row in testdf.iterrows():
                input_raw     = row['image']
                correct_label = row['Label']
                output  = n.query(input_raw)
                label   = np.argmax(output)
                if (label == correct_label):
                    scorecard.append(1)
                else:
                    scorecard.append(0)
                    pass
                pass

            performance = np.asarray(scorecard).sum() / np.asarray(scorecard).size
            newdf = pd.DataFrame({'hidden_node_count' : h,
                                      'learning_rate' : L,
                                               'epoch': e,
                                         'performance': ["%.5f" % performance]})
            NNperformance = NNperformance.append(newdf,  sort=True, ignore_index=True)[newdf.columns.tolist()]
            print(time.time() - start_time)
            pass
        pass
    pass


In [ ]:
NNperformance